In [1]:
# This Python file is meant to run on the Kaggle BigQuery integration

import numpy as np 
import pandas as pd 

import bq_helper
import matplotlib.pyplot as plt
ny_data_set = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="new_york")

[]
Using Kaggle's public dataset BigQuery integration.


## Sample queries from NYC Open Data
The following are sample queries usig NYC Open Data to demonstrate basic understanding of SQL using an open platform. 

In [2]:
# MAPLE SYRUP problem - Find the number of red, black, and sugar maple trees in NYC.
maple_query = """
SELECT 
spc_common AS Tree_type, 
COUNT(spc_common) AS Total
FROM `bigquery-public-data.new_york.tree_census_2015`
WHERE status = 'Alive' AND 
spc_common IN ('sugar maple', 'red maple', 'black maple')
GROUP BY spc_common
ORDER BY Total DESC
"""
maples = ny_data_set.query_to_pandas_safe(maple_query)
maples

,Tree_type,Total
0,red maple,17246
1,sugar maple,2844
2,black maple,114


So the red maple is clearly most common in NYC. These maple types can produce maple syurp, if they are fairly close together one could potentially bottle the syurp. 

__________________________________________________________________________________________________________________________

In [3]:
# The longest citi bike rides! Start and End station. Trip duration is in seconds. 
# 86400 is the conversion from seconds to days
citi_query = """ 
        
        SELECT 
        round((MAX(tripduration) / 86400),2) AS longest_ride_days,
        start_station_name AS start_station
        FROM `bigquery-public-data.new_york.citibike_trips`
        GROUP BY start_station_name
        HAVING MAX(tripduration) > 4000000 
        ORDER BY MAX(tripduration) DESC
        LIMIT 10
"""
citi_trip_one = ny_data_set.query_to_pandas_safe(citi_query)
citi_query_two = """ 
        
        SELECT 
        round((MAX(tripduration) / 86400),2) AS longest_ride_days,
        end_station_name AS end_station
        FROM `bigquery-public-data.new_york.citibike_trips`
        GROUP BY end_station_name
        HAVING MAX(tripduration) > 4000000 
        ORDER BY MAX(tripduration) DESC
        LIMIT 9
"""
citi_trip_two = ny_data_set.query_to_pandas_safe(citi_query_two)


In [4]:
# There are some mismatched ride lengths as compared to start/end station. Removing those below.
citi_trip_one = citi_trip_one.drop(axis = 0, index=5).reset_index(drop=True)

In [5]:
final_df = citi_trip_one.join(citi_trip_two, how = 'inner', lsuffix='_overlap') 
final_df.drop(labels="longest_ride_days_overlap", axis = 1, inplace=True)
final_df

,start_station,longest_ride_days,end_station
0,Fulton St & Grand Ave,77.63,Broadway & Whipple St
1,Cleveland Pl & Spring St,72.35,FDR Drive & E 35 St
2,Vesey Pl & River Terrace,71.49,W 106 St & Amsterdam Ave
3,Central Park S & 6 Ave,70.07,W 74 St & Columbus Ave
4,Lexington Ave & E 24 St,67.88,NYCBS Depot - SSP
5,Kent Ave & S 11 St,62.69,Wythe Ave & Metropolitan Ave
6,South St & Whitehall St,59.89,Christopher St & Greenwich St
7,Forsyth St & Broome St,52.14,NYCBS Depot - PIT
8,E 58 St & 3 Ave,52.04,Rivington St & Ridge St


These are the 8 longest citi bike rides with cooresponding start and end locations. 

In [6]:
# Most dangerous borough for pedestrians since 2015 
peds_query = """
SELECT 
SUM(number_of_pedestrians_killed) AS Total_deaths,
borough
FROM `bigquery-public-data.new_york.nypd_mv_collisions`
WHERE EXTRACT(YEAR FROM timestamp) > 2015
GROUP BY borough
ORDER BY SUM(number_of_pedestrians_killed) DESC
"""
ped_death = ny_data_set.query_to_pandas_safe(peds_query)
ped_death

,Total_deaths,borough
0,111,
1,51,MANHATTAN
2,43,QUEENS
3,40,BROOKLYN
4,23,BRONX
5,6,STATEN ISLAND


111 unassigned boroughs - how do the police not know what borough they are in?

In [7]:
peds_query_two = """
SELECT 
borough, 
zip_code, on_street_name,
off_street_name, 
cross_street_name,
latitude
FROM `bigquery-public-data.new_york.nypd_mv_collisions`
WHERE EXTRACT(YEAR FROM timestamp) > 2015 AND 
borough NOT IN ('MANHATTAN', 'BROOKLYN', 'STATEN ISLAND', 'QUEENS', 'BRONX')
AND number_of_pedestrians_killed > 0
LIMIT 20
"""
ped_death_two = ny_data_set.query_to_pandas_safe(peds_query_two)
ped_death_two

,borough,zip_code,on_street_name,off_street_name,cross_street_name,latitude
0,,None,,,,40.863886
1,,None,MEEKER AVENUE,,,40.726852
2,,None,,7900 17 AVENUE,,NaN
3,,None,VAN WYCK EXPWY,,,40.701763
4,,None,BROADWAY,,,NaN
5,,None,CROSS BAY BOULEVARD,,149 AVENUE,40.671196
6,,None,GRAND CONCOURSE,,,NaN
7,,None,,,,40.755444
8,,None,NOSTRAND AVENUE,,,40.694843
9,,None,,,,40.826523


It looks like it is possible to determine many of the boroughs based on the given data. Using GPS position, or street names could yield almost all of the unassigned borough fields. 

In [8]:
#All the noise complaints since 2015 - When we moved in to 22 Irving Place 
address_query = """WITH bus_noise AS
                    (
                        SELECT 
                            incident_address, 
                            complaint_type, 
                            created_date
                        FROM `bigquery-public-data.new_york.311_service_requests`
                        WHERE complaint_type like '%Noise%' AND EXTRACT(YEAR FROM created_date) > 2015
                    )
                    SELECT 
                           count(complaint_type) AS count, 
                           incident_address AS address
                    FROM bus_noise
                    WHERE incident_address like '%IRVING PLACE%'
                    GROUP BY address
                    HAVING count >= 5
                    ORDER BY count DESC
"""
noise = ny_data_set.query_to_pandas_safe(address_query)
noise

,count,address
0,128,56 IRVING PLACE
1,40,40 IRVING PLACE
2,27,1 IRVING PLACE
3,17,55 IRVING PLACE
4,13,71 IRVING PLACE
5,8,IRVING PLACE
6,7,33 IRVING PLACE
7,5,67 IRVING PLACE
8,5,22 IRVING PLACE


I really wanted 1 Irving Place to have the most, but it seems that someone is very good about calling 311 around 56/55 Irving Place though I do not think that address is nearly as noisy as 1 Irving Place. 